# 第9章: 事前学習済み言語モデル（BERT型）

本章では、BERT型の事前学習済みモデルを利用して、マスク単語の予測や文ベクトルの計算、評判分析器（ポジネガ分類器）の構築に取り組む。

In [1]:
!pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## 80. トークン化

"The movie was full of incomprehensibilities."という文をトークンに分解し、トークン列を表示せよ。

In [2]:
from transformers import BertTokenizer

# 1. トークナイザの準備
# 事前学習済みモデル 'bert-base-uncased' のためのトークナイザをロードします。
# 'bert-base-uncased' は、英語のテキストをすべて小文字に変換してからトークン化します。
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. トークン化する文
sentence = "The movie was full of incomprehensibilities."

# 3. トークン化の実行
# tokenizer.tokenize() メソッドを使って、文をトークンに分割します。
# BERTのトークナイザは、文を単語単位で分割するだけでなく、
# 辞書にない単語や頻度の低い単語をさらに細かいサブワード（例: "incomprehensibilities" -> "in", "##com", "##pre", "##hens", "##ibilities"）
# に分割することがあります。これは "WordPiece" と呼ばれるトークン化手法です。
# "##" は、そのトークンが前のトークンに続く部分であることを示します（つまり、単語の途中）。
tokens = tokenizer.tokenize(sentence)

# 4. 結果の表示
print(f"元の文: {sentence}")
print(f"トークン列: {tokens}")

# 参考: トークンをIDに変換する場合
# BERTモデルに入力する際には、トークンを数値IDに変換する必要があります。
# tokenizer.convert_tokens_to_ids() を使います。
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"トークンID列: {token_ids}")

# 参考: 特殊トークンを含めてIDに変換する場合
# BERTに入力するためには、文の先頭に[CLS]トークン、文末に[SEP]トークンを付加し、
# さらにパディングやアテンションマスクも考慮する必要があります。
# tokenizer.encode_plus() や tokenizer() を使うとこれらを一度に行えます。
encoded_input = tokenizer(sentence) # encode_plus と同様の機能を持つ
print(f"エンコード結果 (input_ids, token_type_ids, attention_mask): {encoded_input}")
print(f"デコードされたトークン (確認用): {tokenizer.convert_ids_to_tokens(encoded_input['input_ids'])}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

元の文: The movie was full of incomprehensibilities.
トークン列: ['the', 'movie', 'was', 'full', 'of', 'inc', '##omp', '##re', '##hen', '##si', '##bilities', '.']
トークンID列: [1996, 3185, 2001, 2440, 1997, 4297, 25377, 2890, 10222, 5332, 14680, 1012]
エンコード結果 (input_ids, token_type_ids, attention_mask): {'input_ids': [101, 1996, 3185, 2001, 2440, 1997, 4297, 25377, 2890, 10222, 5332, 14680, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
デコードされたトークン (確認用): ['[CLS]', 'the', 'movie', 'was', 'full', 'of', 'inc', '##omp', '##re', '##hen', '##si', '##bilities', '.', '[SEP]']


## 81. マスクの予測

"The movie was full of [MASK]."の"[MASK]"を埋めるのに最も適切なトークンを求めよ。

In [3]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

# 1. トークナイザとモデルの準備
# 'bert-base-uncased' を使用します。
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval() # モデルを評価モードに設定（推論時には通常これを行います）

# 2. [MASK] を含む入力文
text = "The movie was full of [MASK]."

# 3. 入力文の準備とトークン化
# tokenizer() を使って、トークン化、IDへの変換、特殊トークンの付加を一度に行います。
# return_tensors="pt" は、PyTorchのテンソル形式で結果を返すよう指定します。
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs['input_ids']

# 4. [MASK]トークンの位置特定
# input_ids の中で、tokenizer.mask_token_id（[MASK]トークンのID）と一致する位置を探します。
# torch.where() は条件に一致する要素のインデックスを返します。
# 結果はタプルのタプルなので、[1]で列インデックスのテンソルを取得し、.item()で数値に変換します。
mask_token_indices = torch.where(input_ids == tokenizer.mask_token_id)[1]
if len(mask_token_indices) == 0:
    print(f"エラー: 文中に {tokenizer.mask_token} が見つかりませんでした。")
    exit()
mask_token_index = mask_token_indices[0].item() # 最初の [MASK] トークンのインデックス

# 5. モデルによる予測
# 勾配計算をしないように torch.no_grad() のコンテキストで実行します（推論時にはメモリ効率が良くなります）。
with torch.no_grad():
    outputs = model(**inputs) # **inputs は {'input_ids': tensor(...), 'attention_mask': tensor(...), ...} を展開して渡します
    predictions = outputs.logits # 予測結果のロジット（スコア）を取得します。形状は (バッチサイズ, シーケンス長, 語彙サイズ)

# 6. [MASK]位置の予測結果抽出
# predictions[0, mask_token_index, :] で、バッチの最初の要素（0）、
# [MASK]トークンの位置（mask_token_index）、全ての語彙に対するロジットを取得します。
mask_token_logits = predictions[0, mask_token_index, :]

# 最もスコアの高いトークンをいくつか取得します。ここでは上位5つとします。
top_k = 5
top_k_tokens_ids = torch.topk(mask_token_logits, top_k, dim=0).indices.tolist()
# dim=0 は、最後の次元（語彙次元）に沿ってトップkを探すことを意味します。
# .indices でIDを取得し、.tolist()でPythonのリストに変換します。

# 7. 予測されたトークンを表示
print(f"入力文: {text}")
print(f"\n予測された {tokenizer.mask_token} のトークン (上位{top_k}位):")
for token_id in top_k_tokens_ids:
    token = tokenizer.convert_ids_to_tokens(token_id)
    print(f"- {token}")

# 参考: [MASK]位置の各候補トークンとそのロジット（スコア）
# print("\n詳細なロジット (上位5位):")
# top_k_logits_values = torch.topk(mask_token_logits, top_k, dim=0).values.tolist()
# for token_id, logit_value in zip(top_k_tokens_ids, top_k_logits_values):
# token = tokenizer.convert_ids_to_tokens(token_id)
# print(f"- トークン: {token}, ID: {token_id}, ロジット: {logit_value:.4f}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


入力文: The movie was full of [MASK].

予測された [MASK] のトークン (上位5位):
- fun
- surprises
- drama
- stars
- laughs


## 82. マスクのtop-k予測

"The movie was full of [MASK]."の"[MASK]"に埋めるのに適切なトークン上位10個と、その確率（尤度）を求めよ。

In [4]:
import torch.nn.functional as F # ソフトマックス関数を使用するためにインポート

# ソフトマックス関数を適用して確率に変換
# dim=0 は、この1次元テンソル（語彙次元）に沿ってソフトマックスを計算することを意味します。
probabilities = F.softmax(mask_token_logits, dim=0)

# 6. 上位10個のトークンとその確率を取得
top_k = 10
# torch.topk は (値, インデックス) のタプルを返します。
top_k_probs_values, top_k_tokens_ids = torch.topk(probabilities, top_k, dim=0)

top_k_tokens_ids_list = top_k_tokens_ids.tolist()
top_k_probs_values_list = top_k_probs_values.tolist()

# 7. 予測されたトークンと確率を表示
print(f"入力文: {text}")
print(f"\n予測された {tokenizer.mask_token} のトークンと確率 (上位{top_k}位):")
print("-" * 40) # 見やすいように区切り線
for token_id, prob_value in zip(top_k_tokens_ids_list, top_k_probs_values_list):
    token = tokenizer.convert_ids_to_tokens(token_id)
    # f-string で {:<15} は左寄せ15文字幅、{:.2%} は小数点以下2桁のパーセンテージ表示
    print(f"トークン: {token:<15} | 確率: {prob_value:.2%}")
print("-" * 40)

入力文: The movie was full of [MASK].

予測された [MASK] のトークンと確率 (上位10位):
----------------------------------------
トークン: fun             | 確率: 10.71%
トークン: surprises       | 確率: 6.63%
トークン: drama           | 確率: 4.47%
トークン: stars           | 確率: 2.72%
トークン: laughs          | 確率: 2.54%
トークン: action          | 確率: 1.95%
トークン: excitement      | 確率: 1.90%
トークン: people          | 確率: 1.83%
トークン: tension         | 確率: 1.50%
トークン: music           | 確率: 1.46%
----------------------------------------


## 83. CLSトークンによる文ベクトル

以下の文の全ての組み合わせに対して、最終層の[CLS]トークンの埋め込みベクトルを用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


In [6]:
import torch
from transformers import BertTokenizer, BertModel
from itertools import combinations # 全ての組み合わせを生成するために使用
import torch.nn.functional as F    # コサイン類似度を計算するために使用

# 1. トークナイザとモデルの準備
# 'bert-base-uncased' を使用します。
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval() # モデルを評価モードに設定

# 2. [CLS]トークンの埋め込みベクトルを取得する関数
def get_cls_embedding(text, tokenizer, model):
    """
    与えられたテキストからBERTの最終層の[CLS]トークンの埋め込みベクトルを取得する。
    Args:
        text (str): 入力テキスト。
        tokenizer: 事前学習済みのBERTトークナイザ。
        model: 事前学習済みのBERTモデル。
    Returns:
        torch.Tensor: [CLS]トークンの埋め込みベクトル。
    """
    # テキストをトークン化し、PyTorchテンソルとしてIDを取得
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # モデルにデバイスを合わせる場合 (GPU使用時など)
    # inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 勾配計算を無効化して推論を実行
    with torch.no_grad():
        outputs = model(**inputs)

    # 最終層の隠れ状態を取得 (outputs.last_hidden_state)
    # 形状: (バッチサイズ, シーケンス長, 隠れ層の次元数)
    # [CLS]トークンはシーケンスの最初のトークンなので、インデックス0でアクセス
    cls_embedding = outputs.last_hidden_state[0, 0, :] # バッチサイズ1なので最初の要素、最初のトークン
    return cls_embedding

# 3. 文のリスト
sentences = [
    "The movie was full of fun.",         # 文0
    "The movie was full of excitement.",  # 文1
    "The movie was full of crap.",        # 文2
    "The movie was full of rubbish."      # 文3
]

# 4. 事前に各文の[CLS]ベクトルを計算して辞書に保存（効率化のため）
print("各文の[CLS]埋め込みベクトルを計算中...")
cls_embeddings = {}
for i, sentence in enumerate(sentences):
    cls_embeddings[i] = get_cls_embedding(sentence, tokenizer, model)
    # print(f"文{i}のベクトル計算完了") # デバッグ用
print("ベクトル計算完了。\n")

# 5. 全ての組み合わせでコサイン類似度を計算し、表示
print("文のペアごとのコサイン類似度:")
print("-" * 60)

# itertools.combinations を使って、重複なしのペアを生成 (例: (0,1), (0,2), (1,2))
for i, j in combinations(range(len(sentences)), 2):
    sentence1 = sentences[i]
    sentence2 = sentences[j]

    embedding1 = cls_embeddings[i]
    embedding2 = cls_embeddings[j]

    # コサイン類似度の計算
    # F.cosine_similarity は2つの1次元テンソル間の類似度を計算する場合、dim=0 を指定
    similarity = F.cosine_similarity(embedding1, embedding2, dim=0)

    print(f"文A: \"{sentence1}\"")
    print(f"文B: \"{sentence2}\"")
    print(f"コサイン類似度: {similarity.item():.4f}") # .item()でテンソルからPythonの数値を取得
    print("-" * 60)

各文の[CLS]埋め込みベクトルを計算中...
ベクトル計算完了。

文のペアごとのコサイン類似度:
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of excitement."
コサイン類似度: 0.9881
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of crap."
コサイン類似度: 0.9558
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of rubbish."
コサイン類似度: 0.9475
------------------------------------------------------------
文A: "The movie was full of excitement."
文B: "The movie was full of crap."
コサイン類似度: 0.9541
------------------------------------------------------------
文A: "The movie was full of excitement."
文B: "The movie was full of rubbish."
コサイン類似度: 0.9487
------------------------------------------------------------
文A: "The movie was full of crap."
文B: "The movie was full of rubbish."
コサイン類似度: 0.9807
-------------------------------------------------------

## 84. 平均による文ベクトル

以下の文の全ての組み合わせに対して、最終層の埋め込みベクトルの平均を用いてコサイン類似度を求めよ。

- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."

In [7]:
import torch
from transformers import BertTokenizer, BertModel
from itertools import combinations
import torch.nn.functional as F

# 1. トークナイザとモデルの準備
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval() # モデルを評価モードに設定

# 2. 平均プーリングされた埋め込みベクトルを取得する関数
def get_mean_pooled_embedding(text, tokenizer, model):
    """
    与えられたテキストからBERTの最終層の埋め込みベクトルの平均を取得する。
    パディングトークンは計算から除外する。
    Args:
        text (str): 入力テキスト。
        tokenizer: 事前学習済みのBERTトークナイザ。
        model: 事前学習済みのBERTモデル。
    Returns:
        torch.Tensor: 平均プーリングされた埋め込みベクトル。
    """
    # テキストをトークン化し、PyTorchテンソルとしてIDとattention_maskを取得
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    attention_mask = inputs['attention_mask'] # パディングを無視するために使用

    # モデルにデバイスを合わせる場合 (GPU使用時など)
    # inputs = {k: v.to(model.device) for k, v in inputs.items()}
    # attention_mask = attention_mask.to(model.device)

    # 勾配計算を無効化して推論を実行
    with torch.no_grad():
        outputs = model(**inputs)

    # 最終層の隠れ状態を取得
    # last_hidden_state の形状: (バッチサイズ, シーケンス長, 隠れ層の次元数)
    last_hidden_state = outputs.last_hidden_state

    # 平均プーリングの計算
    # attention_maskを (バッチサイズ, シーケンス長, 1) に拡張してブロードキャスト可能にする
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()

    # マスクを適用して、パディング部分のベクトルを0にする
    # その後、実際のトークン（マスクが1の部分）のベクトルだけを合計
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, dim=1) # (バッチサイズ, 隠れ層の次元数)

    # 実際のトークン数を計算 (パディングを除いたトークン数)
    # attention_mask の各行の合計が、その文の実際のトークン数
    actual_num_tokens = torch.clamp(attention_mask.sum(dim=1), min=1e-9) # (バッチサイズ)

    # 合計を実際のトークン数で割り、平均を求める
    # actual_num_tokens を (バッチサイズ, 1) に拡張してブロードキャスト可能にする
    mean_pooled_embeddings = sum_embeddings / actual_num_tokens.unsqueeze(1) # (バッチサイズ, 隠れ層の次元数)

    # バッチサイズは1なので、最初の要素（唯一の要素）を返す
    return mean_pooled_embeddings[0]

# 3. 文のリスト
sentences = [
    "The movie was full of fun.",         # 文0
    "The movie was full of excitement.",  # 文1
    "The movie was full of crap.",        # 文2
    "The movie was full of rubbish."      # 文3
]

# 4. 事前に各文の平均プーリングベクトルを計算
print("各文の平均プーリング埋め込みベクトルを計算中...")
mean_pooled_embeddings_dict = {}
for i, sentence in enumerate(sentences):
    mean_pooled_embeddings_dict[i] = get_mean_pooled_embedding(sentence, tokenizer, model)
print("ベクトル計算完了。\n")

# 5. 全ての組み合わせでコサイン類似度を計算し、表示
print("文のペアごとのコサイン類似度 (平均プーリング):")
print("-" * 60)

for i, j in combinations(range(len(sentences)), 2):
    sentence1 = sentences[i]
    sentence2 = sentences[j]

    embedding1 = mean_pooled_embeddings_dict[i]
    embedding2 = mean_pooled_embeddings_dict[j]

    # コサイン類似度の計算
    similarity = F.cosine_similarity(embedding1, embedding2, dim=0)

    print(f"文A: \"{sentence1}\"")
    print(f"文B: \"{sentence2}\"")
    print(f"コサイン類似度: {similarity.item():.4f}")
    print("-" * 60)

各文の平均プーリング埋め込みベクトルを計算中...
ベクトル計算完了。

文のペアごとのコサイン類似度 (平均プーリング):
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of excitement."
コサイン類似度: 0.9568
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of crap."
コサイン類似度: 0.8490
------------------------------------------------------------
文A: "The movie was full of fun."
文B: "The movie was full of rubbish."
コサイン類似度: 0.8169
------------------------------------------------------------
文A: "The movie was full of excitement."
文B: "The movie was full of crap."
コサイン類似度: 0.8352
------------------------------------------------------------
文A: "The movie was full of excitement."
文B: "The movie was full of rubbish."
コサイン類似度: 0.7938
------------------------------------------------------------
文A: "The movie was full of crap."
文B: "The movie was full of rubbish."
コサイン類似度: 0.9226
-------------------------------------------

## 85. データセットの準備

[General Language Understanding Evaluation (GLUE)](https://gluebenchmark.com/) ベンチマークで配布されている[Stanford Sentiment Treebank (SST)](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip) から訓練セット（train.tsv）と開発セット（dev.tsv）のテキストと極性ラベルと読み込み、さらに全てのテキストはトークン列に変換せよ。

In [8]:
!pip install datasets transformers torch

In [10]:
%%capture
!pip install scikit-learn
!pip install numpy
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [12]:
import pandas as pd
from transformers import BertTokenizer

# 1. BERTトークナイザの準備
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. SST-2データセットの読み込み
# ファイルパスは環境に合わせてください。例: 'SST-2/train.tsv'
try:
    train_df = pd.read_csv("SST-2/train.tsv", sep='\t')
    dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t')
except FileNotFoundError:
    print("エラー: SST-2/train.tsv または SST-2/dev.tsv が見つかりません。")
    print("ファイルパスが正しいか、ファイルが存在するか確認してください。")
    print("例: カレントディレクトリに 'SST-2' というフォルダがあり、その中に 'train.tsv' と 'dev.tsv' がある想定です。")
    exit()


# 3. 訓練セットと開発セットのテキストをトークン化
# pandas Series の .apply() メソッドを使って、各文をトークン化します。
print("訓練セットをトークン化しています...")
train_df['tokens'] = train_df['sentence'].apply(lambda text: tokenizer.tokenize(text))
print("訓練セットのトークン化完了。")

print("開発セットをトークン化しています...")
dev_df['tokens'] = dev_df['sentence'].apply(lambda text: tokenizer.tokenize(text))
print("開発セットのトークン化完了。\n")


# 4. 結果の確認 (最初の数件を表示)
def print_dataframe_sample(dataset_name, df, num_samples=3):
    print(f"--- {dataset_name} (最初の{num_samples}件) ---")
    # DataFrameが短い場合に備えて、num_samples と len(df) の小さい方までループ
    for i in range(min(num_samples, len(df))):
        row = df.iloc[i] # i番目の行を取得
        print(f"例文 {i+1}:")
        print(f"  テキスト: {row['sentence']}")
        print(f"  ラベル: {row['label']} (0: negative, 1: positive)")
        print(f"  トークン: {row['tokens']}") # 'tokens'列から直接リストを取得
        print("-" * 20)
    print("\n")

print_dataframe_sample("訓練セット (トークン化済み)", train_df, num_samples=3)
print_dataframe_sample("開発セット (トークン化済み)", dev_df, num_samples=3)

# 参考: データセットの構造（DataFrameの情報）
print("--- 訓練データセットの情報 ---")
train_df.info()
print("\n--- 開発データセットの情報 ---")
dev_df.info()

訓練セットをトークン化しています...
訓練セットのトークン化完了。
開発セットをトークン化しています...
開発セットのトークン化完了。

--- 訓練セット (トークン化済み) (最初の3件) ---
例文 1:
  テキスト: hide new secretions from the parental units 
  ラベル: 0 (0: negative, 1: positive)
  トークン: ['hide', 'new', 'secret', '##ions', 'from', 'the', 'parental', 'units']
--------------------
例文 2:
  テキスト: contains no wit , only labored gags 
  ラベル: 0 (0: negative, 1: positive)
  トークン: ['contains', 'no', 'wit', ',', 'only', 'labor', '##ed', 'gag', '##s']
--------------------
例文 3:
  テキスト: that loves its characters and communicates something rather beautiful about human nature 
  ラベル: 1 (0: negative, 1: positive)
  トークン: ['that', 'loves', 'its', 'characters', 'and', 'communicate', '##s', 'something', 'rather', 'beautiful', 'about', 'human', 'nature']
--------------------


--- 開発セット (トークン化済み) (最初の3件) ---
例文 1:
  テキスト: it 's a charming and often affecting journey . 
  ラベル: 1 (0: negative, 1: positive)
  トークン: ['it', "'", 's', 'a', 'charming', 'and', 'often', 'affecting', 'journey', 

## 86. ミニバッチの作成

85で読み込んだ訓練データの一部（例えば冒頭の4事例）に対して、パディングなどの処理を行い、トークン列の長さを揃えてミニバッチを構成せよ。

In [13]:
import pandas as pd
from transformers import BertTokenizer
import torch # PyTorchをインポート

# 0. 事前準備 (前回のコードからの続きを想定)
# train_df が既に読み込まれ、'sentence' 列を持っているとします。
# もし train_df がない場合は、まず前回の処理を実行してください。
# 例としてダミーの train_df を作成しますが、実際は前回の結果を使用してください。
if 'train_df' not in locals():
    print("注意: train_df が定義されていません。ダミーデータで続行します。")
    data = {'sentence': ["hide new secretions from the parental units",
                         "contains no wit , only labored gags",
                         "that loves its characters and communicates something rather beautiful about human nature",
                         "remains utterly satisfied to remain the same throughout"],
            'label': [0, 0, 1, 0]}
    train_df = pd.DataFrame(data)
    # 'tokens' 列は今回は直接使用しませんが、前回の処理では作成されていました。
    # train_df['tokens'] = train_df['sentence'].apply(lambda text: tokenizer.tokenize(text))


# 1. BERTトークナイザの準備 (既にロード済みと仮定)
if 'tokenizer' not in locals():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. 訓練データから冒頭4事例のテキストを取得
num_examples_for_batch = 4
if len(train_df) < num_examples_for_batch:
    print(f"エラー: train_df の事例数が {num_examples_for_batch} 未満です。")
    exit()

raw_texts_batch = train_df['sentence'].head(num_examples_for_batch).tolist()
print("ミニバッチに使用するテキスト:")
for i, text in enumerate(raw_texts_batch):
    print(f"  例文{i+1}: {text}")
print("-" * 30)

# 3. トークナイザによるミニバッチのエンコード
# tokenizer() にテキストのリストを渡すと、必要な前処理（特殊トークン追加、ID化、パディング、アテンションマスク作成）をまとめて行います。
encoded_batch = tokenizer(
    raw_texts_batch,
    padding=True,           # バッチ内の最も長いシーケンスに合わせてパディングします ('longest' と同等)
    truncation=True,        # モデルの最大長を超える場合は切り詰めます
    return_tensors="pt",    # PyTorchテンソルで結果を返します
    add_special_tokens=True # [CLS], [SEP] を自動で追加します
)

# encoded_batch は辞書形式で、以下のキーを含みます:
# 'input_ids': トークンIDのテンソル
# 'attention_mask': アテンションマスクのテンソル
# 'token_type_ids': トークンタイプIDのテンソル (今回は単一文なので全て0)

input_ids = encoded_batch['input_ids']
attention_mask = encoded_batch['attention_mask']
# token_type_ids はSST-2のような単一文分類では必須ではない場合もありますが、取得しておきます。
token_type_ids = encoded_batch.get('token_type_ids') # .get() で存在しない場合もエラーにしない

# 4. ミニバッチの内容を表示
print("\n--- ミニバッチの構成要素 ---")
print("Input IDs (トークンID):")
print(input_ids)
print(f"形状: {input_ids.shape}") # (バッチサイズ, シーケンス長)
print("-" * 30)

print("Attention Mask (アテンションマスク):")
print(attention_mask)
print(f"形状: {attention_mask.shape}") # (バッチサイズ, シーケンス長)
print("-" * 30)

if token_type_ids is not None:
    print("Token Type IDs (トークンタイプID):")
    print(token_type_ids)
    print(f"形状: {token_type_ids.shape}") # (バッチサイズ, シーケンス長)
    print("-" * 30)

# 参考: 最初の事例のトークンとIDとマスクを確認
print("\n--- 最初の事例の詳細 ---")
example_index = 0
print(f"元のテキスト: {raw_texts_batch[example_index]}")

# IDをトークンに戻して確認
tokens_from_ids = tokenizer.convert_ids_to_tokens(input_ids[example_index])
print(f"IDから復元したトークン: {tokens_from_ids}")

print(f"対応するInput IDs: {input_ids[example_index].tolist()}") # .tolist() でテンソルをリストに変換して表示
print(f"対応するAttention Mask: {attention_mask[example_index].tolist()}")
if token_type_ids is not None:
    print(f"対応するToken Type IDs: {token_type_ids[example_index].tolist()}")

ミニバッチに使用するテキスト:
  例文1: hide new secretions from the parental units 
  例文2: contains no wit , only labored gags 
  例文3: that loves its characters and communicates something rather beautiful about human nature 
  例文4: remains utterly satisfied to remain the same throughout 
------------------------------

--- ミニバッチの構成要素 ---
Input IDs (トークンID):
tensor([[  101,  5342,  2047,  3595,  8496,  2013,  1996, 18643,  3197,   102,
             0,     0,     0,     0,     0],
        [  101,  3397,  2053, 15966,  1010,  2069,  4450,  2098, 18201,  2015,
           102,     0,     0,     0,     0],
        [  101,  2008,  7459,  2049,  3494,  1998, 10639,  2015,  2242,  2738,
          3376,  2055,  2529,  3267,   102],
        [  101,  3464, 12580,  8510,  2000,  3961,  1996,  2168,  2802,   102,
             0,     0,     0,     0,     0]])
形状: torch.Size([4, 15])
------------------------------
Attention Mask (アテンションマスク):
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 

## 87. ファインチューニング

訓練セットを用い、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。

In [2]:
!pip install evaluate

In [1]:
!pip install --upgrade transformers

In [16]:
import torch
import pandas as pd
from datasets import Dataset # pandas DataFrame から Dataset オブジェクトを作成するためにインポート
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate # Hugging Face の評価ライブラリ

# 0. デバイスの確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")

# 1. トークナイザの準備
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

# 2. データセットの準備 (pandasでTSVを読み込み、Datasetオブジェクトに変換)
print("ローカルTSVファイルからSST-2データセットを読み込んでいます...")
train_file_path = "SST-2/train.tsv"
dev_file_path = "SST-2/dev.tsv"

try:
    train_df = pd.read_csv(train_file_path, sep='\t', header=0) # header=0 で1行目をヘッダーとして使用
    dev_df = pd.read_csv(dev_file_path, sep='\t', header=0)
except FileNotFoundError:
    print(f"エラー: {train_file_path} または {dev_file_path} が見つかりません。")
    print("ファイルパスが正しいか、ファイルが指定の場所に存在するか確認してください。")
    print("（カレントディレクトリに 'SST-2' フォルダがあり、その中にTSVファイルがある想定です）")
    exit()

# pandas DataFrame を Hugging Face Dataset オブジェクトに変換
# SST-2のTSVは通常 'sentence' と 'label' カラムを持っています
train_dataset_hf = Dataset.from_pandas(train_df)
eval_dataset_hf = Dataset.from_pandas(dev_df)

print(f"訓練データ: {len(train_dataset_hf)}件, 検証データ: {len(eval_dataset_hf)}件 読み込み完了。")

# トークン化関数
def tokenize_function(examples):
    # examples["sentence"] は文のリスト（バッチ処理時）または単一の文
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

print("データセットをトークン化しています...")
tokenized_train_dataset = train_dataset_hf.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset_hf.map(tokenize_function, batched=True)

# Trainerで利用するために不要な列を削除し、ラベル列名を'labels'に変更
# 元の 'sentence' 列はトークン化後は不要
# DataFrameのインデックスが '__index_level_0__' として残っている場合があるので、それも削除
columns_to_remove = ["sentence"]
if "__index_level_0__" in tokenized_train_dataset.column_names:
    columns_to_remove.append("__index_level_0__")

tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(columns_to_remove)

# 'label' 列を 'labels' にリネーム (Trainerが期待する列名)
if 'label' in tokenized_train_dataset.column_names:
    tokenized_train_dataset = tokenized_train_dataset.rename_column("label", "labels")
if 'label' in tokenized_eval_dataset.column_names:
    tokenized_eval_dataset = tokenized_eval_dataset.rename_column("label", "labels")


# PyTorchテンソル形式に設定 (Trainerが必要とする列を指定)
# 'labels' 列もテンソル形式にする必要がある
columns_for_format = ["input_ids", "attention_mask", "labels"]
tokenized_train_dataset.set_format("torch", columns=columns_for_format)
tokenized_eval_dataset.set_format("torch", columns=columns_for_format)


# 3. モデルの準備
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

# 4. 評価指標の準備
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# 5. 訓練引数の設定
training_args = TrainingArguments(
    output_dir="./results_sst2_from_tsv", # 出力ディレクトリ名を変更
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs_sst2_from_tsv', # ログディレクトリ名も変更
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

# 6. Trainer の初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset, # ここで準備したデータセットを使用
    eval_dataset=tokenized_eval_dataset,   # ここで準備したデータセットを使用
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 7. ファインチューニングの実行
print("ファインチューニングを開始します...")
trainer.train()
print("ファインチューニング完了。")

# 8. 検証セットでの評価
print("検証セットで評価を実行します...")
eval_results = trainer.evaluate()

print(f"\n--- 検証結果 ---")
print(f"正解率 (Accuracy): {eval_results['eval_accuracy']:.4f}")
for key, value in eval_results.items():
    if key != "eval_accuracy": # eval_accuracyは既表示なのでそれ以外
        print(f"{key}: {value}")

# (オプション) モデル保存
# trainer.save_model("./fine_tuned_sst2_model_from_tsv")
# tokenizer.save_pretrained("./fine_tuned_sst2_model_from_tsv")
# print("\nファインチューニングされたモデルとトークナイザを保存しました。")

使用デバイス: cuda
ローカルTSVファイルからSST-2データセットを読み込んでいます...
訓練データ: 67349件, 検証データ: 872件 読み込み完了。
データセットをトークン化しています...


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

## 88. 極性分析

問題87でファインチューニングされたモデルを用いて、以下の文の極性を予測せよ。

- "The movie was full of incomprehensibilities."
- "The movie was full of fun."
- "The movie was full of excitement."
- "The movie was full of crap."
- "The movie was full of rubbish."


## 89. アーキテクチャの変更

問題87とは異なるアーキテクチャ（例えば[CLS]トークンを用いるか、各トークンの最大値プーリングを用いるなど）の分類モデルを設計し、事前学習済みモデルを極性分析タスク向けにファインチューニングせよ。検証セット上でファインチューニングされたモデルの正解率を計測せよ。